In [ ]:
!pip install python-dotenv --upgrade --quiet langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 4.5 MB/s eta 0:00:00


In [ ]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

Enter your Groq API Key: ··········


In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

MODEL_CONFIG = {
    "technical": """
You are a senior software engineer support assistant.
Give debugging steps, explain errors, and show corrected code.
Be concise and precise.
""",

    "billing": """
You are a customer billing support agent.
Be empathetic and explain refund/payment policies clearly.
Never discuss programming or technical debugging.
""",

    "general": """
You are a friendly helpful assistant for casual conversation.
Keep the tone natural and conversational.
"""
}

In [ ]:
def route_prompt(user_input: str) -> str:

    router_prompt = """
Classify the user message into ONE category:

technical → programming, errors, debugging, code
billing → payments, refunds, subscription charges
general → greetings or casual conversation
tool → live data requests like bitcoin price, weather, time

Return ONLY one word:
technical
billing
general
tool
"""

    response = llm.invoke([
        SystemMessage(content=router_prompt),
        HumanMessage(content=user_input)
    ])

    return response.content.strip().lower()

In [ ]:
import random

def tool_expert(user_input):
    if "bitcoin" in user_input.lower():
        price = round(random.uniform(58000, 72000), 2)
        return f"Current Bitcoin price is approximately ${price} USD."
    return "I can fetch live data, but I don't recognize the requested metric."

In [ ]:
expert_llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7)

In [ ]:
def process_request(user_input: str):

    category = route_prompt(user_input)
    print(f"\n[Router] → {category.upper()} expert selected\n")

    if category == "tool":
        return tool_expert(user_input)

    system_prompt = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])

    response = expert_llm.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_input)
    ])

    return response.content

In [ ]:
while True:
    user_input = input("\nUser: ")

    if user_input.lower() in ["exit", "quit"]:
        break

    reply = process_request(user_input)
    print("\nAssistant:", reply)


User: My python script is throwing an IndexError on line 5.

[Router] → TECHNICAL expert selected


Assistant: To troubleshoot the error, I'll need more information about your script. However, I can guide you through a general process to identify and fix the `IndexError`.

**Common scenarios that cause IndexError:**

1.  Accessing an element in a list or array with an index that is out of range.
2.  Using a negative index that points to a position before the start of the list.

**Step-by-Step Debugging:**

1.  **Check the code:** Inspect the line of code causing the error (line 5 in your case). Look for any indexing operations.
2.  **Verify list/array length:** Ensure that the list or array you're trying to access has at least as many elements as the index you're using.
3.  **Check negative indexing:** If you're using negative indexing, ensure that it's not pointing to a position before the start of the list.

**Example Code:**

Suppose you have a list `fruits` and you're trying to ac